In [2]:
import folium

import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from IPython.display import display
from shapely.geometry import Point, Polygon
from concurrent.futures import ThreadPoolExecutor

In [2]:
mt_coordinates = [
    [45, -113.547],
    [45, -104.04],
    [49.001, -104.04],
    [49.001, -116.047]
    ]

ks_coordinates = [
    [36.993, -102.041],
    [36.9986, -94.618],
    [40.003, -94.602],
    [40.0034, -102.0518]
    ]

nd_coordinates = [
    [45.94, -104.05],
    [45.94, -96.55],
    [49.00, -97.25],
    [49.00, -104.05]
    ]



mt_poly = Polygon(mt_coordinates)
ks_poly = Polygon(ks_coordinates)
nd_poly = Polygon(nd_coordinates)


def state(row):
    point = Point(row['Latitude'], row['Longitude'])
    return mt_poly.contains(point) or ks_poly.contains(point) or nd_poly.contains(point)


def filter(df):
    df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')
    threshold = pd.Timestamp('2001-09-20')
    temp = df[(df['DATE'] > threshold) & (df['TMAX'].notna()) & (df['TMIN'].notna())]
    print(f'{df.index[-1]+1} rows processed')
    
    return temp

In [3]:
chunks = pd.read_csv('weather_data_us.csv', chunksize=1000000)
executor = ThreadPoolExecutor()
futures = [executor.submit(filter, c) for c in chunks]

1000000 rows processed
2000000 rows processed
3000000 rows processed
4000000 rows processed
5000000 rows processed
6000000 rows processed
7000000 rows processed
8000000 rows processed
9000000 rows processed
10000000 rows processed
11000000 rows processed
12000000 rows processed
13000000 rows processed
14000000 rows processed
15000000 rows processed
16000000 rows processed
17000000 rows processed
18000000 rows processed
19000000 rows processed
20000000 rows processed
21000000 rows processed
22000000 rows processed
23000000 rows processed
24000000 rows processed
25000000 rows processed
26000000 rows processed
27000000 rows processed
28000000 rows processed
29000000 rows processed
30000000 rows processed
31000000 rows processed
32000000 rows processed
33000000 rows processed
34000000 rows processed
35000000 rows processed
36000000 rows processed
37000000 rows processed
38000000 rows processed
39000000 rows processed
40000000 rows processed
41000000 rows processed
42000000 rows processed
4

155840906 rows processed


In [4]:
i = 1
for f in futures:
    results = pd.DataFrame(f.result())
    results.to_csv(f'usa_temp_data/chunk_{i}.csv')
    print(f'chunk {i} done')
    i += 1

chunk 1 done
chunk 2 done
chunk 3 done
chunk 4 done
chunk 5 done
chunk 6 done
chunk 7 done
chunk 8 done
chunk 9 done
chunk 10 done
chunk 11 done
chunk 12 done
chunk 13 done
chunk 14 done
chunk 15 done
chunk 16 done
chunk 17 done
chunk 18 done
chunk 19 done
chunk 20 done
chunk 21 done
chunk 22 done
chunk 23 done
chunk 24 done
chunk 25 done
chunk 26 done
chunk 27 done
chunk 28 done
chunk 29 done
chunk 30 done
chunk 31 done
chunk 32 done
chunk 33 done
chunk 34 done
chunk 35 done
chunk 36 done
chunk 37 done
chunk 38 done
chunk 39 done
chunk 40 done
chunk 41 done
chunk 42 done
chunk 43 done
chunk 44 done
chunk 45 done
chunk 46 done
chunk 47 done
chunk 48 done
chunk 49 done
chunk 50 done
chunk 51 done
chunk 52 done
chunk 53 done
chunk 54 done
chunk 55 done
chunk 56 done
chunk 57 done
chunk 58 done
chunk 59 done
chunk 60 done
chunk 61 done
chunk 62 done
chunk 63 done
chunk 64 done
chunk 65 done
chunk 66 done
chunk 67 done
chunk 68 done
chunk 69 done
chunk 70 done
chunk 71 done
chunk 72 done
c

In [3]:
files = glob('usa_temp_data/*')
i = 0
j = 0
retain = pd.read_csv(files[0])
retain.drop(retain.index, inplace=True)

for f in files:
    data = pd.read_csv(f)
    df = data[data.apply(state, axis=1)]
    retain = pd.concat([retain, df])
    i += 1
    if i > 9:
        i = 0
        j += 1
        retain.reset_index(inplace=True)
        retain.drop(['index', 'Unnamed: 0'], axis=1, inplace=True)
        retain.to_csv(f'mt_nd_ks_temp_data/chunk_{j}')
        print(f'chunk_{j} saved')

chunk_1 saved
chunk_2 saved
chunk_3 saved
chunk_4 saved
chunk_5 saved
chunk_6 saved
chunk_7 saved
chunk_8 saved
chunk_9 saved
chunk_10 saved
chunk_11 saved
chunk_12 saved
chunk_13 saved
chunk_14 saved
chunk_15 saved


In [15]:
map_center = [39.5, -98.35]  
map = folium.Map(location=map_center, zoom_start=5)

nd_coordinates = [
    [45.94, -104.05],
    [45.94, -96.55],
    [49.00, -97.25],
    [49.00, -104.05]
]
folium.Polygon(
    locations=nd_coordinates,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.3,
    popup='North Dakota'
).add_to(map)

# Define coordinates for Kansas (approximate bounding box)
ks_coordinates = [
    [36.993, -102.041],
    [36.9986, -94.618],
    [40.003, -94.602],
    [40.0034, -102.0518]
]
folium.Polygon(
    locations=ks_coordinates,
    color='green',
    fill=True,
    fill_color='green',
    fill_opacity=0.3,
    popup='Kansas'
).add_to(map)

# Define coordinates for Montana (approximate bounding box)
mt_coordinates = [
    [45, -113.547],
    [45, -104.04],
    [49.001, -104.04],
    [49.001, -116.047]
]
folium.Polygon(
    locations=mt_coordinates,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.3,
    popup='Montana'
).add_to(map)


display(map)
map.save('map.html')


In [10]:
files = glob('mt_nd_ks_temp_data/*')
final_data = pd.DataFrame(columns=['DATE', 'TMAX', 'TMIN', 'PRCP'])

for f in files:
    data = pd.read_csv(f)
    data_group = data.groupby('DATE').agg({'TMAX':'mean', 'TMIN':'mean', 'PRCP':'mean'})
    data_group.reset_index(inplace=True)
    final_data = pd.concat([final_data, data_group])

final_data = final_data.groupby('DATE').agg({'TMAX':'mean', 'TMIN':'mean', 'PRCP':'mean'})
final_data.reset_index(inplace=True)
final_data.to_csv('usa_temp.csv', header=True)
final_data.head()

C:\Users\X Warrior\AppData\Local\Temp\ipykernel_19180\153392308.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([final_data, data_group])


,DATE,TMAX,TMIN,PRCP
0,2001-09-21,238.365049,68.575360,4.253906
1,2001-09-22,235.377840,70.167401,4.074222
2,2001-09-23,232.480929,57.344989,19.256044
3,2001-09-24,243.705465,57.121888,0.350587
4,2001-09-25,249.911855,54.301195,1.080603
